In [1]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

import matplotlib.image as mpimg
from keras import backend as K

import skimage.io
import matplotlib.pyplot as plt
import cv2
import glob
from skimage.transform import resize
import keras
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input, decode_predictions
import numpy as np
from tqdm import tqdm

import keras
from keras.applications.xception import Xception

from keras.utils import to_categorical

%matplotlib inline
K.clear_session()

In [2]:

model = InceptionV3(weights='imagenet')
model.compile(loss=keras.losses.categorical_crossentropy, optimizer= keras.optimizers.SGD(lr=0.01),metrics = ['accuracy'])

In [3]:
num_classes=1000

file = open("drive/My Drive/Tesi/Colab Notebooks/ILSVRC2014_clsloc_validation_ground_truth.txt","r")
label = file.readlines()
file.close()
label =[int(item) for item in label]

In [4]:
import scipy.io
meta = scipy.io.loadmat("drive/My Drive/Tesi/Colab Notebooks/meta_clsloc.mat")
original_idx_to_synset = {}
synset_to_name = {}
for i in range(1000):
    ilsvrc2014_id = int(meta["synsets"][0,i][0][0][0])
    synset = meta["synsets"][0,i][1][0]
    name = meta["synsets"][0,i][2][0]
    original_idx_to_synset[ilsvrc2014_id] = synset
    synset_to_name[synset] = name
synset_to_keras_idx = {}
keras_idx_to_name = {}
f = open("drive/My Drive/Tesi/Colab Notebooks/synset_words.txt","r")
idx = 0
for line in f:
    parts = line.split(" ")
    synset_to_keras_idx[parts[0]] = idx
    keras_idx_to_name[idx] = " ".join(parts[1:])
    idx += 1
f.close()
def convert_original_idx_to_keras_idx(idx):
    return synset_to_keras_idx[original_idx_to_synset[idx]]

In [5]:
label = np.array([convert_original_idx_to_keras_idx(idx) for idx in label])
label = to_categorical(label , num_classes)

In [ ]:
pbar = tqdm(total=50000)
img_rows=299; img_cols=299; img_channels=3;
x_val = []
y_val = []
test_accuracy_list = []
test_loss_list = []
j=1
i=1
k=0
m=1
for filename in sorted(glob.glob("drive/My Drive/Tesi/Colab Notebooks/ILSVRC2014_images_val/*.JPEG")):
    img = np.float32(skimage.io.imread(filename))
    img = resize(img, (img_rows, img_cols), order=1)
    if img.shape!=(299,299,3):
      img = np.stack((img,img,img), -1)
    #img = np.reshape(img, (1,img.shape[0],img.shape[1],img.shape[2]))
    img = preprocess_input(img)

    x_val.append(img)
    if i==2500:
      #for item in x_val:
        #if item.shape!=(299,299,3):
          #print("Shape prima:",item.shape)
          #item = np.stack((item,item,item), -1)
          #print("Shape dopo:",item.shape)
      #print("Dimensione x_val:",len(x_val))

      x_val = np.reshape(x_val , (2500,img_rows,img_cols,img_channels))
      #x_val = np.float32(x_val) / 255
      y_val=label[k:(k+2500)]
      #for item in x_val:
          #plt.figure(); plt.imshow(item/255); plt.show();
    
      #print("Dimensione x_val:",len(x_val))
      #print("Dimensione y_val:",len(y_val))
      test_loss, test_accuracy=model.evaluate(x_val, y_val, verbose=True)
      test_loss_list.append(test_loss)
      test_accuracy_list.append(test_accuracy)
      print('Test loss:',test_loss)
      print('Test accuracy:',test_accuracy)
      x_val = []
      y_val = []
      i=0
      j=j+1
      k=k+2500
    
    i=i+1
    m=m+1
    if m==40001:
      accuracy_media=sum(test_accuracy_list) / float(len(test_accuracy_list)) 
      print("Accuracy media:",accuracy_media)

    pbar.update()

pbar.close()
accuracy_media=sum(test_accuracy_list) / float(len(test_accuracy_list)) 
print("Accuracy media:",accuracy_media)

  5%|▍         | 2496/50000 [01:46<24:14, 32.66it/s]

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


  5%|▌         | 2503/50000 [02:08<15:35:48,  1.18s/it]

Test loss: 0.9894868461608887
Test accuracy: 0.7732


 10%|█         | 5000/50000 [03:52<12:39:06,  1.01s/it]

...

 90%|█████████ | 45000/50000 [6:50:28<5:52:22,  4.23s/it]

Test loss: 1.0022630046844483
Test accuracy: 0.764


 95%|█████████▌| 47500/50000 [7:14:03<2:59:18,  4.30s/it]

Test loss: 0.9533987060070038
Test accuracy: 0.7804


 99%|█████████▉| 49744/50000 [7:35:42<02:11,  1.95it/s]